# Will data & metadata stay in sync?

Here, we walk through different errors that can occur while saving artifacts & metadata records, and show that the LaminDB instance does not get corrupted by dangling metadata or artifacts.

Transactions within Python across data & metadata are [ACID](https://en.wikipedia.org/wiki/ACID).

If an upload process is externally killed and Python cannot run clean-up operations anymore, the artifact is internally flagged with `artifact._storage_completed = False`. This is visible on the UI. You can then re-run `lamin save` or `artifact.save()` to attempt uploading the artifact a second time.

In [ ]:
!lamin init --storage ./test-acid

In [ ]:
import pytest
import lamindb as ln
from upath import UPath

## Save error due to failed upload

Let's try to save an artifact to a storage location without permission.

In [ ]:
artifact = ln.Artifact.from_anndata(
    ln.examples.datasets.anndata_mouse_sc_lymph_node(),
    description="Mouse Lymph Node scRNA-seq",
)

Because the public API only allows you to set a default storage for which you have permission, we need to hack it:

In [ ]:
ln.settings.storage._root = UPath("s3://nf-core-awsmegatests")

This raises a `PermissionError`:

In [ ]:
ln.settings.verbosity = "debug"

In [ ]:
with pytest.raises(PermissionError) as error:
    artifact.save()
print(error.exconly())

Let's now check that no metadata records were added to the database:

In [ ]:
assert len(ln.Artifact.filter()) == 0

## Save error during bulk creation

In [ ]:
filepath = ln.examples.datasets.file_jpg_paradisi05()
artifact = ln.Artifact(filepath, description="My image")
artifacts = [artifact, "this is not a record"]

This raises an exception:

In [ ]:
with pytest.raises(Exception) as error:
    ln.save(artifacts)
print(error.exconly())

Nothing got saved:

In [ ]:
artifacts = ln.Artifact.filter()
assert len(artifacts) == 0

If a list of data objects is passed to `ln.save()` and the upload of one of these data objects fails, the successful uploads are maintained and a `RuntimeError` is raised, listing the successfully uploaded data objects up until that point.

In [ ]:
!lamin delete --force test-acid